# Coleta e Análise de Tweets com a API do Twitter (X)

Este trabalho tem como objetivo demonstrar a coleta, limpeza e análise de tweets em tempo real usando a API oficial do Twitter, com foco em um tema de interesse. Ao final, são feitas análises simples de conteúdo e sentimentos.

Tema escolhido: **Esportes**


## 1. Coleta dos tweets

Nesta etapa, utilizei a API do Twitter (v2) com um Bearer Token para buscar os 50 tweets mais recentes contendo a palavra-chave **"Esportes"**.

A requisição foi feita utilizando a biblioteca `requests`, com parâmetros definidos para limitar o número de resultados e garantir o uso eficiente da API.


In [ ]:
import requests

bearer_token = "AAAAAAAAAAAAAAAAAAAAAMLc0AEAAAAA2Q2iwlSRfHonSJk%2BxjDFsP1ZX8c%3DxxeyplHGIAlYyh5rd1gzMFW2r3SyKNEU6JC536DUbWRIegikLN"

# URL de busca
def montar_url_busca(query):
    base_url = "https://api.twitter.com/2/tweets/search/recent"
    return f"{base_url}?query={query}&max_results=50&tweet.fields=lang"

# Função para obter tweets
def obter_tweets(url, bearer):
    headers = {
        "Authorization": f"Bearer {bearer}"
    }
    resposta = requests.get(url, headers=headers)
    if resposta.status_code != 200:
        raise Exception(f"Erro: {resposta.status_code}\n{resposta.text}")
    return resposta.json()

tema = "esportes" 
url = montar_url_busca(tema)
tweets = obter_tweets(url, bearer_token)

# Mostrar os textos dos tweets
if "data" in tweets:
    for tweet in tweets["data"]:
        print(tweet["text"])
else:
    print("Nenhum tweet encontrado.")


Jorge Jesus, ex-Flamengo, contratado na Seleção Brasileira tem resposta de Neymar
https://t.co/44getyjiSC
@guilhermedias Ei, @guilhermedias, com base nos seus posts, você parece se alinhar a uma figura conservadora, possivelmente Jair Bolsonaro. Sua crítica a Lula e interação com Joice Hasselmann, apoiadora de Bolsonaro, sugerem uma tendência à direita. Apesar do foco em esportes, seus comentários https://t.co/fojVr3QS46
@iamgomesmatheus Olá, Matheus! Pelos seus posts, você parece com uma figura política ligada à tecnologia e esportes, como Luiz Philippe de Orleans e Bragança. Seu foco em tech (ex.: Huawei vs iPhone) e futebol, além de interações com perfis de direita, sugere uma inclinação conservadora. Mas, https://t.co/rLErsJKD5W
@douglasperdigao Ei, @douglasperdigao! Sem acesso total aos seus tuítes, é difícil cravar, mas pelo seu português e possíveis laços com esportes (talvez futebol 7), você pode lembrar uma figura pública brasileira como Neymar, pelo estilo ativo nas redes, ou

## 2. Limpeza dos tweets

Os tweets coletados foram tratados para facilitar a análise textual. Foi removido:

- Links
- Menções a outros usuários (@usuário)
- Hashtags
- Pontuações
- Letras maiúsculas (convertidas para minúsculas)

Essa limpeza me ajudou a evitar ruídos nas contagens e análises posteriores.


In [2]:
import re

def limpar_texto(texto):
    texto = texto.lower()  # minúsculas
    texto = re.sub(r"http\S+", "", texto)  # remove links
    texto = re.sub(r"@\w+", "", texto)  # remove menções
    texto = re.sub(r"#\w+", "", texto)  # remove hashtags
    texto = re.sub(r"[^\w\s]", "", texto)  # remove pontuação
    return texto.strip()

# Aplicar limpeza e mostrar resultados
tweets_limpos = [limpar_texto(tweet["text"]) for tweet in tweets["data"]]

for i, tweet in enumerate(tweets_limpos[:5]):
    print(f"{i+1}: {tweet}")


1: jorge jesus exflamengo contratado na seleção brasileira tem resposta de neymar
2: ei  com base nos seus posts você parece se alinhar a uma figura conservadora possivelmente jair bolsonaro sua crítica a lula e interação com joice hasselmann apoiadora de bolsonaro sugerem uma tendência à direita apesar do foco em esportes seus comentários
3: olá matheus pelos seus posts você parece com uma figura política ligada à tecnologia e esportes como luiz philippe de orleans e bragança seu foco em tech ex huawei vs iphone e futebol além de interações com perfis de direita sugere uma inclinação conservadora mas
4: ei  sem acesso total aos seus tuítes é difícil cravar mas pelo seu português e possíveis laços com esportes talvez futebol 7 você pode lembrar uma figura pública brasileira como neymar pelo estilo ativo nas redes ou tite pela vibe estratégica
5: rt  papo de alto nível no 4d esportes dessa segundafeira rafael sobis e mauri dorneles receberam o volante fernando reges do i


## 3. Análise de frequência de palavras

Após a limpeza, juntei todos os tweets em um único texto e fiz a contagem das palavras mais frequentes, ignorando palavras muito curtas (menos de 4 letras).

Essa etapa mostra quais termos são mais usados nos tweets sobre o tema escolhido.


In [3]:
from collections import Counter

# Juntar todos os tweets limpos em um só texto
texto_unico = " ".join(tweets_limpos)
palavras = texto_unico.split()

# Contar as palavras mais comuns (ignorando palavras pequenas)
contagem = Counter([p for p in palavras if len(p) > 3])
mais_comuns = contagem.most_common(10)

print("Palavras mais comuns:")
for palavra, freq in mais_comuns:
    print(f"{palavra}: {freq}")


Palavras mais comuns:
você: 41
esportes: 34
seus: 24
como: 20
posts: 15
mais: 15
futebol: 14
parece: 11
muito: 11
quer: 9


In [4]:
!pip install -q nltk

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...


## 4. Análise de Sentimentos

Utilizei a biblioteca VADER (`nltk.sentiment.vader`) para aplicar uma análise de sentimentos básica nos tweets. Embora o VADER seja mais preciso para textos em inglês, ele pode oferecer uma noção geral sobre o tom dos tweets mesmo em português.

Cada tweet foi classificado como:

- **Positivo**
- **Negativo**
- **Neutro**

No final, foi feito um resumo quantitativo dos sentimentos encontrados.


In [6]:
# Inicializa o analisador
analisador = SentimentIntensityAnalyzer()

# Função pra classificar o sentimento
def classificar_sentimento(texto):
    pontuacoes = analisador.polarity_scores(texto)
    if pontuacoes["compound"] >= 0.05:
        return "Positivo"
    elif pontuacoes["compound"] <= -0.05:
        return "Negativo"
    else:
        return "Neutro"

# Aplica em cada tweet
resultados = [(tweet, classificar_sentimento(tweet)) for tweet in tweets_limpos]

# Mostra os primeiros resultados
for i, (tweet, sentimento) in enumerate(resultados[:10]):
    print(f"{i+1}: ({sentimento}) {tweet}")


1: (Neutro) jorge jesus exflamengo contratado na seleção brasileira tem resposta de neymar
2: (Neutro) ei  com base nos seus posts você parece se alinhar a uma figura conservadora possivelmente jair bolsonaro sua crítica a lula e interação com joice hasselmann apoiadora de bolsonaro sugerem uma tendência à direita apesar do foco em esportes seus comentários
3: (Neutro) olá matheus pelos seus posts você parece com uma figura política ligada à tecnologia e esportes como luiz philippe de orleans e bragança seu foco em tech ex huawei vs iphone e futebol além de interações com perfis de direita sugere uma inclinação conservadora mas
4: (Neutro) ei  sem acesso total aos seus tuítes é difícil cravar mas pelo seu português e possíveis laços com esportes talvez futebol 7 você pode lembrar uma figura pública brasileira como neymar pelo estilo ativo nas redes ou tite pela vibe estratégica
5: (Negativo) rt  papo de alto nível no 4d esportes dessa segundafeira rafael sobis e mauri dorneles recebera

In [7]:
from collections import Counter

sentimentos = [s for _, s in resultados]
contagem_sentimentos = Counter(sentimentos)

print("\nResumo de sentimentos:")
for tipo, qtd in contagem_sentimentos.items():
    print(f"{tipo}: {qtd}")



Resumo de sentimentos:
Neutro: 37
Negativo: 5
Positivo: 8


## 5. Conclusão

A atividade permitiu entender como textos curtos e informais como tweets podem ser analisados para revelar padrões e sentimentos.

No tema **esporte**, os tweets apresentaram uma maioria de sentimentos neutros, mas também foram identificados tons positivos e negativos.
